In [1]:
import sqlite3
import os
import hashlib
import binascii

In [3]:
# access_database requires the name of a sqlite3 database file and the query.
# It does not return the result of the query.
def access_database(dbfile, query):
    connect = sqlite3.connect(dbfile)
    cursor = connect.cursor()
    cursor.execute(query)
    connect.commit()
    connect.close()
    
# access_database requires the name of a sqlite3 database file and the query.
# It returns the result of the query
def access_database_with_result(dbfile, query):
    connect = sqlite3.connect(dbfile)
    cursor = connect.cursor()
    rows = cursor.execute(query).fetchall()
    connect.commit()
    connect.close()
    return rows

In [7]:
def hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')

In [8]:
#Pre-poluate the credential table with 10 username and hashed password
def user_entry(dbfile):
    for i in range(1,11):
        user= 'test'+str(i)
        password = 'password'+str(i)
        hash_p = hash_password(password)
        access_database(dbfile,'INSERT INTO credential VALUES ("%s", "%s")'%(user,hash_p))

In [9]:
# Set up the tables

def setup_assessment_tables(dbfile):
    # Get rid of any existing data
    access_database(dbfile, "DROP TABLE IF EXISTS credential")
    access_database(dbfile, "DROP TABLE IF EXISTS login")
    access_database(dbfile, "DROP TABLE IF EXISTS session")
    access_database(dbfile, "DROP TABLE IF EXISTS vehicle")
    access_database(dbfile, "DROP TABLE IF EXISTS v_type")
    
    
    # Freshly setup tables
    access_database(dbfile, "CREATE TABLE credential (username TEXT, password TEXT)")
    access_database(dbfile, "CREATE TABLE login (username TEXT, magic TEXT,starttime DATETIME, endtime DATETIME)")
    access_database(dbfile, "CREATE TABLE session (username TEXT, magic TEXT)")
    access_database(dbfile, "CREATE TABLE vehicle (username TEXT,magic TEXT,Mode TEXT, Location TEXT, occupancy INTEGER,Type TEXT, time DATETIME)")
    access_database(dbfile, "CREATE TABLE v_type (Type TEXT)")
    
    # Populate the tables with some initial data
    user_entry(dbfile)
    access_database(dbfile, "INSERT INTO v_type VALUES ('car'),('taxi'),('bus'),('motorbike'),('bicycle'),('van'),('truck'),('other')")

In [10]:
setup_assessment_tables("initial_db.db") # for server.py

In [11]:
setup_assessment_tables("offline_db.db")  #for task8 and 9

#### Few example entries 

In [8]:
access_database_with_result("initial_db.db", "SELECT * from login")

[('test1', '6465206688', '2020-12-03 21:37', '2020-12-03 21:38'),
 ('test', '1234567890', '2020-12-03 21:38', '2020-12-03 21:38'),
 ('test', '1234567890', '2020-12-03 21:38', '2020-12-03 21:38'),
 ('test5', '34851015595', '2020-12-03 21:39', '2020-12-03 21:39')]

In [9]:
access_database_with_result("initial_db.db", "SELECT * from vehicle")

[('test1', '6465206688', 'add', 'london', 1, 'car', '2020-12-03 21:37'),
 ('test1', '6465206688', 'add', 'bath', 3, 'taxi', '2020-12-03 21:37'),
 ('test1', '6465206688', 'undo', 'bath', 2, 'truck', '2020-12-03 21:37'),
 ('test1', '6465206688', 'add', 'bath', 1, 'bicycle', '2020-12-03 21:37'),
 ('test5', '34851015595', 'add', 'bristol', 3, 'van', '2020-12-03 21:39')]

In [13]:
access_database_with_result("offline_db.db", "SELECT * from vehicle")

[('test1', None, 'add', 'main street', 1, 'car', '2020-06-01 09:05'),
 ('test1', None, 'add', 'ring road', 4, 'bus', '2020-06-01 10:10'),
 ('test1', None, 'add', 'main street', 2, 'car', '2020-06-01 11:11'),
 ('test1', None, 'undo', 'main street', 1, 'car', '2020-06-01 12:11'),
 ('test1', None, 'add', 'main street', 1, 'car', '2020-06-02 15:10'),
 ('test1', None, 'add', 'main street', 2, 'taxi', '2020-06-02 15:12'),
 ('test1', None, 'add', 'main street', 4, 'taxi', '2020-06-02 15:13'),
 ('test1', None, 'add', 'ring road', 3, 'car', '2020-06-02 15:12'),
 ('test1', None, 'undo', 'main street', 2, 'taxi', '2020-06-02 15:15'),
 ('test1', None, 'add', 'busy road', 2, 'bus', '2020-06-03 11:34'),
 ('test1', None, 'add', 'busy road', 3, 'bus', '2020-06-03 11:35'),
 ('test1', None, 'add', 'university avenue', 2, 'bicycle', '2020-06-03 11:36'),
 ('test1', None, 'add', 'busy road', 1, 'bus', '2020-06-03 11:38'),
 ('test1', None, 'add', 'busy road', 4, 'bus', '2020-06-03 11:41'),
 ('test1', None, 

In [14]:
access_database_with_result("offline_db.db", "SELECT * from login")

[('test8', None, '2020-05-10 09:00', '2020-05-10 10:00'),
 ('test7', None, '2020-06-11 09:00', '2020-06-11 10:00'),
 ('test6', None, '2020-06-19 09:00', '2020-06-19 10:00'),
 ('test6', None, '2020-06-19 11:00', '2020-06-19 12:00'),
 ('test6', None, '2020-06-19 13:00', '2020-06-19 14:00'),
 ('test6', None, '2020-06-19 15:00', '2020-06-19 16:01'),
 ('test5', None, '2020-06-20 12:47', '2020-06-20 13:47'),
 ('test5', None, '2020-06-21 09:23', '2020-06-21 11:23'),
 ('test5', None, '2020-07-09 09:00', '2020-07-09 11:12'),
 ('test7', None, '2020-07-10 09:00', '2020-07-10 10:01'),
 ('test4', None, '2020-07-11 19:10', '2020-07-11 21:11'),
 ('test4', None, '2020-07-14 15:24', '2020-07-14 16:25'),
 ('test5', None, '2020-07-14 19:10', '2020-07-14 21:41'),
 ('test3', None, '2020-07-04 08:31', '2020-07-04 13:44'),
 ('test2', None, '2020-07-12 11:30', '2020-07-12 17:32'),
 ('test1', None, '2020-07-14 12:00', '2020-07-14 16:25'),
 ('test10', None, '2020-07-16 06:00', '2020-07-16 13:00')]